In [ ]:
# Imports

import numpy as np
import logging; logger = logging.getLogger(__name__)


from opendrift.models.oceandrift import OceanDrift, Lagrangian3DArray
from opendrift.models.leeway import Leeway # For Search and Rescure. May not be relevant
from opendrift.models.plastdrift import PlastDrift
from opendrift.models.sedimentdrift import SedimentDrift # Looks most promising
from opendrift.models.radionuclides import Radionuclide

from opendrift.readers import reader_netCDF_CF_generic

from pprint import pprint
from opendrift.readers import reader_global_landmask

from datetime import datetime

from opendrift.models.basemodel import OpenDriftSimulation
from opendrift.elements import LagrangianArray


### Define the model

Here, we define the class *GhostNetElement*, which is supposed to represent a single ghost net in the ocean. Then, the class *GhostNetDrift* handles the simulation. <br>

As of now, I simply copied the code from `opendrift.models.sedimentdrift.SedimentDrift`. We should also check out `opendrift.models.radionnuclides.Radionnuclide` as it also incorporates diameter, density, etc. Then, the RadionNuclideDrift shows how these properties can be handled in the simulation. <br>

The most important parts here are the `variables` field in GhostNetElement, the `required_variables` field in GhostNetDrift and the `update()` function in GhostNetDrift.

In [ ]:
class GhostNetElement(Lagrangian3DArray):
    variables = Lagrangian3DArray.add_variables([
        ('settled', {'dtype': np.uint8,  # 0 is active, 1 is settled
                     'units': '1',
                     'default': 0}),
        ('terminal_velocity', {'dtype': np.float32,
                               'units': 'm/s',
                               'default': -0.001})  # 1 mm/s negative buoyancy
        ])

# Differences between RadionNuclide and Sediment:
# diameter in radionuclide: BUT only needed to update terminal velocity (don't include)
# density: -------||---------


class GhostNetDrift(OceanDrift):
    """Model for sediment drift, under development
    """

    ElementType = GhostNetElement

    required_variables = {
        'x_sea_water_velocity': {'fallback': 0},
        'y_sea_water_velocity': {'fallback': 0},
        'upward_sea_water_velocity': {'fallback': 0},
        'x_wind': {'fallback': 0},
        'y_wind': {'fallback': 0},
        'sea_surface_wave_stokes_drift_x_velocity': {'fallback': 0},
        'sea_surface_wave_stokes_drift_y_velocity': {'fallback': 0},
        'sea_surface_wave_period_at_variance_spectral_density_maximum': {'fallback': 0},
        'sea_surface_wave_mean_period_from_variance_spectral_density_second_frequency_moment': {'fallback': 0},
        'land_binary_mask': {'fallback': None},
        'ocean_vertical_diffusivity': {'fallback': 0.02},
        'ocean_mixed_layer_thickness': {'fallback': 50},
        'sea_floor_depth_below_sea_level': {'fallback': 0},
        }

    def __init__(self, *args, **kwargs):
        """ Constructor of SedimentDrift module
        """

        super(GhostNetDrift, self).__init__(*args, **kwargs)

        # By default, sediments do not strand towards coastline
        # TODO: A more sophisticated stranding algorithm is needed
        self._set_config_default('general:coastline_action', 'previous')

        # Vertical mixing is enabled as default
        self._set_config_default('drift:vertical_mixing', True)

        # To allow stranding particles
        self._set_config_default('general:coastline_action', 'stranding')

    def update(self):
        """Update positions and properties of sediment particles.
        """

        # Advecting here all elements, but want to soon add
        # possibility of not moving settled elements, until
        # they are resuspended. May then need to send a boolean
        # array to advection methods below
        self.advect_ocean_current()

        self.vertical_advection()

        self.advect_wind()  # Wind shear in upper 10cm of ocean

        self.stokes_drift()

        #self.vertical_mixing()  # Including buoyancy and settling

        self.resuspension()

    def bottom_interaction(self, seafloor_depth):
        """Sub method of vertical_mixing, determines settling"""
        # Elements at or below seafloor are settled, by setting
        # self.elements.moving to 0.
        # These elements will not move until eventual later resuspension.
        settling = np.logical_and(self.elements.z <= seafloor_depth, self.elements.moving==1)
        if np.sum(settling) > 0:
            logger.debug('Settling %s elements at seafloor' % np.sum(settling))
            self.elements.moving[settling] = 0

    def resuspension(self):
        """Resuspending elements if current speed > .5 m/s"""
        resuspending = np.logical_and(self.current_speed()>.5, self.elements.moving==0)
        if np.sum(resuspending) > 0:
            # Allow moving again
            self.elements.moving[resuspending] = 1
            # Suspend 1 cm above seafloor
            self.elements.z[resuspending] = self.elements.z[resuspending] + .01


In [ ]:
o = GhostNetDrift(loglevel=0)

# Want to try different models:
# Plastic
#o = PlastDrift()

In [ ]:
pprint(o.required_variables)

### Add Readers. There are two readers of interest: 
- Generic reader that retrieves ocean data for simulating currents
- Landmask reader for land data (where ocean meets land). This file is simply 3D tensor (matrix) containing 1 where there is land and 0 where there is ocean <br>

Each reader needs a filename, that is either a local file or URL. Here I use the URL provided [here](https://opendrift.github.io/tutorial.html#import-a-specific-model-for-the-relevant-application) <br>

**They are accessed in the following manner:**

In [ ]:
# Generic reader
reader_norkyst = reader_netCDF_CF_generic.Reader(
    filename='https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be'
)

# Landmask reader. 
reader_landmask = reader_global_landmask.Reader()

In [ ]:
o.add_reader([reader_landmask, reader_norkyst])

### Getting number and position of particles
Using data for fishing activity over the timespan 2021-2022 to estimate the placement and number of particles released. Count any activity over half an hour, then release another particle for every 10 hours of activity. Can change this number.

In [ ]:
# Want to get the position of the particles from GlobalFishingWatch
import pandas as pd
fishing_activity_21_22 = pd.read_csv("Fishing_activity_2021-2022.csv", delimiter=';', header=0,usecols=[0,1,3,4], low_memory=False)
fa_gill_nets = fishing_activity_21_22[fishing_activity_21_22['geartype'] == 'set_gillnets']
# Filter out fishing activity under 0.5 hours:
fa_gill_nets = fa_gill_nets[fa_gill_nets['Apparent Fishing hours'] > 0.5]
fa_gill_nets['num_particles'] = fa_gill_nets['Apparent Fishing hours']//10+1

In [ ]:
# Only southern Norway (South of Namsos)
fa_gill_nets_south = fa_gill_nets[fa_gill_nets["Lat"]<64]
print(sum(fa_gill_nets_south["num_particles"]))
print(sum(fa_gill_nets["num_particles"]))

In [ ]:
#parts_lists = np.ones(fa_gill_nets_south['num_particles'])
lats = fa_gill_nets_south['Lat']#*np.ones(fa_gill_nets_south['num_particles'])
lats = np.array(lats)
lons = fa_gill_nets_south['Lon']#*np.ones(fa_gill_nets_south['num_particles'])
lons = np.array(lons)
num_parts = fa_gill_nets_south['num_particles']
num_parts = np.array(num_parts)
print(lats)
print(num_parts)

lats_total = []
lons_total = []
for i in range(len(num_parts)):
    for j in range(int(num_parts[i])):
        lats_total.append(lats[i])
        lons_total.append(lons[i])
lats_total = np.array(lats_total)
lons_total = np.array(lons_total)


lats_total and lons_total are the coordinates of the particles being released. Where multiple particles should be placed in the same spot, there are multiples of the same latitude and longitude.

### Seeding elements
To seed an element simply means releasing it into the ocean (for simulation). <br>

**Important:**
- If `lon` and `lat` are given as arrays of two elements, the GhostNet is dropped randomly at the line between them. Here, the radius can incorporate the uncertainty of the line's width. 
- If `time` is given as an array, then the GhostNet is dropped randomly in the timeinterval. 

In [ ]:
# These are the standard arguments. 
# number GhostNet is dropped at longitude 4.3, lattitude 60 within a radius of 1000 meters. 
#o.seed_elements(lon=4.5, lat=60, radius=1000, number=10, time=reader_norkyst.start_time)

# Try to seed with the actual particles
o.seed_elements(lon=lons_total[0:], lat=lats_total[0:], time=reader_norkyst.start_time)

In [ ]:
# Inspect properties of the GhostNet dropped
o.elements_scheduled

### Configurating the simulation

We can also provide custom configurations to the simulation, that may suit GhostNets better.

In [ ]:
# Parameters available to configurate:
o.list_configspec()

### Running the model

In [ ]:
# Variables the simulation needs
o.required_variables

In [ ]:
# Hover over run to see documentation
#o.run(time_step=3600, steps=1)
o.run(time_step=3600,steps=24*100,outfile='sim_100_days.nc')

In [ ]:
o.plot()

Can save the simulation data to file on .nc format.

In [ ]:
# Importing data for simulation instead of running it again.

o = GhostNetDrift(loglevel=0)
o.io_import_file('stranded_100_steps.nc')

In [ ]:

o.plot(show_initial=False)

In [ ]:
# Using netCDF4

import netCDF4 as nc
import matplotlib.pyplot as plt
import plotly.express as px


fn = 'stranded_100_steps.nc'
ds = nc.Dataset(fn)

lons_read = ds.variables['lon'][:]
lats_read = ds.variables['lat'][:]
lons_array = np.array(lons_read)
lats_array = np.array(lats_read)

In [ ]:
fig = px.density_mapbox([lats_array,lons_array], lat='Lat', lon='Lon', z='Congregation of nets',mapbox_style='open-street-map')
fig.update_layout(title = 'Congregation of nets', title_x=0.5)
fig.show()

In [ ]:
plt.figure()
plt.imshow(lats_array,lons_array)
#plt.plot([0,1,2,3],[1,2,3,4])
plt.show()

### Testing SINTEF data (not relevant ATM)

In [ ]:
# Change name of time variable
import os
for file in [f'./data/aver-monthly{bool(i)*f"-{i}"}' for i in range(6)]:
    os.system(f'ncrename -v time,time_var {file}.nc {file}-xr.nc')

In [ ]:
norkyst = reader_netCDF_CF_generic.Reader(
    [f'./data/aver-monthly{bool(i)*f"-{i}"}-xr.nc' for i in range(6)]
)

In [ ]:
print(reader_netCDF_CF_generic.Reader('./data/aver-monthly-xr.nc'))

In [ ]:
print(reader_netCDF_CF_generic.Reader('./data/aver-monthly-1-xr.nc'))

In [ ]:
norkyst = reader_netCDF_CF_generic.Reader(
    './data/aver-monthly-xr.nc', standard_name_mapping={'time_var': 'time'}
)

In [ ]:
reader_norkyst.variable_mapping

In [ ]:
from netCDF4 import Dataset

ds = Dataset('./data/aver-monthly.nc')

type(ds.dimensions.values())

dims = ds.dimensions.values()
time = dims.mapping['time']

In [ ]:
time

In [ ]:
ds['time']

In [ ]:
vars = ds.variables.values()
vars

In [ ]:
reader_norkyst = reader_netCDF_CF_generic.Reader(
    'https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be'
)
print(reader_norkyst)

In [ ]:
reader_norkyst.variables

In [ ]:
from datetime import datetime, timedelta
from opendrift.models.shipdrift import ShipDrift

o = ShipDrift()
o.add_readers_from_list(
    ['https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be']
)
#o.disable_vertical_motion()
o.seed_elements(lon=4.85, lat=60, time=datetime.now(), number=10000, radius=1000)

o.run(duration=timedelta(hours=72))
o.animation(filename='animation.mp4')